## SETUP

#### Import Libraries

In [1]:
#!pip install sqlalchemy
import os
import numpy
import pymysql
import pandas as pd
from sqlalchemy import create_engine, text

In [2]:
import time

import requests
import requests.exceptions

In [3]:
host_name = "localhost"
port = "3306"
user_id = "root"
pwd = "Uva!1819"

src_dbname = "nfl"
dst_dbname = "nfl_dw"

#### Define Functions for Getting Data From and Setting Data Into Databases (Reused from Lab03)

In [4]:
def get_dataframe(user_id, pwd, host_name, db_name, sql_query):
    conn_str = f"mysql+pymysql://{user_id}:{pwd}@{host_name}/{db_name}"
    sqlEngine = create_engine(conn_str, pool_recycle=3600)
    connection = sqlEngine.connect()
    dframe = pd.read_sql(sql_query, connection);
    connection.close()
    
    return dframe


def set_dataframe(user_id, pwd, host_name, db_name, df, table_name, pk_column, db_operation):
    conn_str = f"mysql+pymysql://{user_id}:{pwd}@{host_name}/{db_name}"
    sqlEngine = create_engine(conn_str, pool_recycle=3600)
    connection = sqlEngine.connect()
    
    if db_operation == "insert":
        df.to_sql(table_name, con=connection, index=False, if_exists='replace')
        connection.execute(text(f"ALTER TABLE {table_name} ADD PRIMARY KEY ({pk_column});"))
            
    elif db_operation == "update":
        df.to_sql(table_name, con=connection, index=False, if_exists='append')
    
    connection.close()

#### Define API Request Function with Error Handling

In [5]:
def get_api_response(url, params):
    try:
        response = requests.request("GET", url, params=params)
        response.raise_for_status()

    except requests.exceptions.HTTPError as errh:
        return "An Http Error occurred: " + repr(errh)
    except requests.exceptions.ConnectionError as errc:
        return "An Error Connecting to the API occurred: " + repr(errc)
    except requests.exceptions.Timeout as errt:
        return "A Timeout Error occurred: " + repr(errt)
    except requests.exceptions.RequestException as err:
        return "An Unknown Error occurred: " + repr(err)

    return response.json()

#### Creating Superbowls Data Warehouse 

In [6]:
conn_str = f"mysql+pymysql://{user_id}:{pwd}@{host_name}"
sqlEngine = create_engine(conn_str, pool_recycle=3600)
conn = sqlEngine.connect()

conn.execute(text(f"DROP DATABASE IF EXISTS `{dst_dbname}`;"))
conn.execute(text(f"CREATE DATABASE `{dst_dbname}`;"))
conn.execute(text(f"USE {dst_dbname};"))

# Run date dim SQL script after this

## CREATING DIMENSION TABLES

#### Players table

In [7]:
sql_mvps = "SELECT DISTINCT mvp FROM nfl.superbowls;"
df_players = get_dataframe(user_id, pwd, host_name, src_dbname, text(sql_mvps))
df_players.rename(columns={"mvp":"player_name"}, inplace=True)
df_players.insert(0, 'player_key', numpy.arange(df_players.shape[0]))

df_players.head(5)

,player_key,player_name
0,0,Patrick Mahomes
1,1,Cooper Kupp
2,2,Tom Brady
3,3,Julian Edelman
4,4,Nick Foles


#### Reading NFL Teams Data from CSV

In [8]:
df_teams = pd.read_csv("nfl_teams.csv")

df_teams.head(2)

,Team,Stadium,Location,Capacity,Founded
0,Arizona Cardinals,State Farm Stadium,Glendale,63400,1898
1,Atlanta Falcons,Mercedes-Benz Stadium,Atlanta,71000,1966


#### Stadium Table

Stadiums That Have Hosted a Superbowl

In [9]:
sql_sb_stadiums = "SELECT DISTINCT stadium, city FROM nfl.superbowls;"
df_stadiums = get_dataframe(user_id, pwd, host_name, src_dbname, text(sql_sb_stadiums))
df_stadiums.rename(columns={"stadium":"stadium_name"}, inplace=True)

df_stadiums.head(2)

,stadium_name,city
0,State Farm Stadium,Glendale
1,SoFi Stadium,Inglewood


Stadiums Currently Used

In [10]:
df_current_stadiums = df_teams.copy()[['Stadium', 'Location']].drop_duplicates()
df_current_stadiums.rename(columns={"Stadium":"stadium_name", "Location":"city"}, inplace=True)

df_current_stadiums.head(2)

,stadium_name,city
0,State Farm Stadium,Glendale
1,Mercedes-Benz Stadium,Atlanta


Combining

In [11]:
df_stadiums = pd.merge(df_stadiums, df_current_stadiums, on = ["stadium_name", "city"], how = "outer")
df_stadiums.insert(0, 'stadium_key', numpy.arange(df_stadiums.shape[0]))

df_stadiums.head(2)

,stadium_key,stadium_name,city
0,0,State Farm Stadium,Glendale
1,1,SoFi Stadium,Inglewood


##### Geocoding API

In [12]:
latlon_list = []

url = "https://geocode.maps.co/search?q="

for index, row in df_stadiums.iterrows():
    stadium_name = row[1]
    stadium_city = row[2]
    time.sleep(1)
    # json = get_api_response(f"{url}{stadium_name}, {stadium_city}", "")
    json = get_api_response(f"{url}{stadium_city}", "")
    lat = json[0]['lat']
    lon = json[0]['lon']
    latlon_list.append({"latitude": lat, "longitude": lon})

In [13]:
df_latlon = pd.DataFrame(latlon_list)
df_latlon.head(2)

,latitude,longitude
0,34.1469416,-118.2478471
1,33.9562003,-118.353132


In [14]:
df_stadiums = df_stadiums.join(df_latlon)
df_stadiums.head(2)

,stadium_key,stadium_name,city,latitude,longitude
0,0,State Farm Stadium,Glendale,34.1469416,-118.2478471
1,1,SoFi Stadium,Inglewood,33.9562003,-118.353132


#### Team Table

In [15]:
df_teams.insert(0, 'team_key', numpy.arange(df_teams.shape[0]))
df_teams.rename(columns={"Team":"team_name", "Founded":"year_founded"}, inplace=True)

df_stadiums_lookup = df_stadiums[['stadium_key', 'stadium_name']]
df_teams = pd.merge(df_teams, df_stadiums_lookup, left_on = "Stadium", right_on = "stadium_name", how = "left")

drop_cols = ['Capacity', 'Location', 'Stadium', 'stadium_name']
df_teams.drop(drop_cols, axis=1, inplace=True)

df_teams.head(2)

,team_key,team_name,year_founded,stadium_key
0,0,Arizona Cardinals,1898,0
1,1,Atlanta Falcons,1966,4


#### Load into Data Warehouse

In [16]:
db_operation = "insert"

tables = [('dim_players', df_players, 'player_key'),
         ('dim_stadiums', df_stadiums, 'stadium_key'),
         ('dim_teams', df_teams, 'team_key')]

In [17]:
for table_name, dataframe, primary_key in tables:
    set_dataframe(user_id, pwd, host_name, dst_dbname, dataframe, table_name, primary_key, db_operation)

## CREATING FACT TABLE

#### Creating Dataframe from Superbowls Table

In [18]:
sql_superbowls = "SELECT * FROM nfl.superbowls;"
df_superbowls = get_dataframe(user_id, pwd, host_name, src_dbname, text(sql_superbowls))
df_superbowls.head(2)

,game_date,superbowl_title,winner,winner_points,loser,loser_points,mvp,stadium,city,state
0,2023-02-12,LVII (57),Kansas City Chiefs,38,Philadelphia Eagles,35,Patrick Mahomes,State Farm Stadium,Glendale,Arizona
1,2022-02-13,LVI (56),Los Angeles Rams,23,Cincinnati Bengals,20,Cooper Kupp,SoFi Stadium,Inglewood,California


## Cleaning Data

### Replacing Old Team Names

In [19]:
df_previous_team_names = pd.read_csv("previous_nfl_team_names.csv")
df_previous_team_names.head(2)

,Historical Team Name,Current Team Name
0,Chicago Cardinals,Arizona Cardinals
1,Baltimore Colts,Indianapolis Colts


In [20]:
df_team_names_lookup = pd.DataFrame({"Historical Team Name": df_teams['team_name'], "Current Team Name": df_teams['team_name']})
df_team_names_lookup = pd.concat([df_team_names_lookup, df_previous_team_names], ignore_index=True)

#### Turning winner to current name

In [21]:
df_superbowls = pd.merge(df_superbowls, df_team_names_lookup, left_on = "winner", right_on = "Historical Team Name", how = "left")
drop_cols = ['winner', 'city', 'state', 'Historical Team Name']
df_superbowls.drop(drop_cols, axis=1, inplace=True)

df_superbowls.rename(columns={"Current Team Name": "winner"}, inplace=True)

df_superbowls.head(2)

,game_date,superbowl_title,winner_points,loser,loser_points,mvp,stadium,winner
0,2023-02-12,LVII (57),38,Philadelphia Eagles,35,Patrick Mahomes,State Farm Stadium,Kansas City Chiefs
1,2022-02-13,LVI (56),23,Cincinnati Bengals,20,Cooper Kupp,SoFi Stadium,Los Angeles Rams


#### Turning loser to current name

In [22]:
df_superbowls = pd.merge(df_superbowls, df_team_names_lookup, left_on = "loser", right_on = "Historical Team Name", how = "left")
drop_cols = ['loser', 'Historical Team Name']
df_superbowls.drop(drop_cols, axis=1, inplace=True)

df_superbowls.rename(columns={"Current Team Name": "loser"}, inplace=True)

df_superbowls.head(2)

,game_date,superbowl_title,winner_points,loser_points,mvp,stadium,winner,loser
0,2023-02-12,LVII (57),38,35,Patrick Mahomes,State Farm Stadium,Kansas City Chiefs,Philadelphia Eagles
1,2022-02-13,LVI (56),23,20,Cooper Kupp,SoFi Stadium,Los Angeles Rams,Cincinnati Bengals


## Stadium Lookup

In [23]:
df_superbowls = pd.merge(df_superbowls, df_stadiums_lookup, left_on = "stadium", right_on = "stadium_name", how = "left")
drop_cols = ['stadium', 'stadium_name']
df_superbowls.drop(drop_cols, axis=1, inplace=True)

df_superbowls.head(2)

,game_date,superbowl_title,winner_points,loser_points,mvp,winner,loser,stadium_key
0,2023-02-12,LVII (57),38,35,Patrick Mahomes,Kansas City Chiefs,Philadelphia Eagles,0
1,2022-02-13,LVI (56),23,20,Cooper Kupp,Los Angeles Rams,Cincinnati Bengals,1


## Winning and Losing Team Lookup

In [24]:
df_teams_lookup = df_teams[['team_key', 'team_name']]

df_superbowls = pd.merge(df_superbowls, df_teams_lookup, left_on = "winner", right_on = "team_name", how = "left")
df_superbowls = pd.merge(df_superbowls, df_teams_lookup, left_on = "loser", right_on = "team_name", how = "left")

drop_cols = ['winner', 'loser', 'team_name_x', 'team_name_y']
df_superbowls.drop(drop_cols, axis=1, inplace=True)

df_superbowls.rename(columns={"team_key_x":"winning_team_key", "team_key_y":"losing_team_key"}, inplace=True)

df_superbowls.head(2)

,game_date,superbowl_title,winner_points,loser_points,mvp,stadium_key,winning_team_key,losing_team_key
0,2023-02-12,LVII (57),38,35,Patrick Mahomes,0,15,25
1,2022-02-13,LVI (56),23,20,Cooper Kupp,1,18,6


## MVP Lookup

In [25]:
df_superbowls = pd.merge(df_superbowls, df_players, left_on = "mvp", right_on = "player_name", how = "left")
drop_cols = ['mvp', 'player_name']
df_superbowls.drop(drop_cols, axis=1, inplace=True)

df_superbowls.head(2)

,game_date,superbowl_title,winner_points,loser_points,stadium_key,winning_team_key,losing_team_key,player_key
0,2023-02-12,LVII (57),38,35,0,15,25,0
1,2022-02-13,LVI (56),23,20,1,18,6,1


## Date Lookup

In [27]:
sql_date = "SELECT * FROM nfl_dw.dim_date;"
df_date = get_dataframe(user_id, pwd, host_name, src_dbname, text(sql_date)).astype({"full_date":'datetime64'})[['date_key', 'full_date']]

df_date.head(2)

,date_key,full_date
0,19670101,1967-01-01
1,19670102,1967-01-02


## Adding Weather Data (API Call)

In [28]:
url = "https://archive-api.open-meteo.com/v1/archive"
daily_values = {"temperature_2m_max", "temperature_2m_min", "precipitation_sum", "windspeed_10m_max"}

In [29]:
weather = []
    
for index, row in df_superbowls.iterrows():
    stadium_key = row[7]
    
    stadium = df_stadiums.loc[stadium_key]
    params = {"latitude": stadium.latitude, "longitude": stadium.longitude, "start_date": df_superbowls['game_date'].iloc[0], "end_date": df_superbowls['game_date'].iloc[0], 
              "daily": daily_values, "timezone": "auto"}
    weather_json = get_api_response(url, params)
    weather.append({"temperature_2m_max": weather_json['daily']['temperature_2m_max'][0],
                     "temperature_2m_min": weather_json['daily']['temperature_2m_min'][0],
                     "precipitation_sum": weather_json['daily']['precipitation_sum'][0],
                     "windspeed_10m_max": weather_json['daily']['windspeed_10m_max'][0]})

In [30]:
df_superbowls['game_date'] = pd.to_datetime(df_superbowls['game_date'])
df_superbowls = pd.merge(df_superbowls, df_date, left_on = "game_date", right_on="full_date")

drop_cols = ['game_date', 'full_date']
df_superbowls.drop(drop_cols, axis=1, inplace=True)

df_superbowls.head(2)

,superbowl_title,winner_points,loser_points,stadium_key,winning_team_key,losing_team_key,player_key,date_key
0,LVII (57),38,35,0,15,25,0,20230212
1,LVI (56),23,20,1,18,6,1,20220213


In [31]:
df_weather = pd.DataFrame(weather)
df_weather = df_weather.astype(float)
df_weather.head(2)

,temperature_2m_max,temperature_2m_min,precipitation_sum,windspeed_10m_max
0,14.3,5.9,0.0,15.4
1,14.7,5.8,0.0,25.6


In [32]:
df_superbowls = df_superbowls.join(df_weather)
df_superbowls.head(2)

,superbowl_title,winner_points,loser_points,stadium_key,winning_team_key,losing_team_key,player_key,date_key,temperature_2m_max,temperature_2m_min,precipitation_sum,windspeed_10m_max
0,LVII (57),38,35,0,15,25,0,20230212,14.3,5.9,0.0,15.4
1,LVI (56),23,20,1,18,6,1,20220213,14.7,5.8,0.0,25.6


## Primary Key

In [33]:
df_superbowls.insert(0, 'superbowl_key', numpy.arange(df_superbowls.shape[0]))
df_superbowls.head(2)

,superbowl_key,superbowl_title,winner_points,loser_points,stadium_key,winning_team_key,losing_team_key,player_key,date_key,temperature_2m_max,temperature_2m_min,precipitation_sum,windspeed_10m_max
0,0,LVII (57),38,35,0,15,25,0,20230212,14.3,5.9,0.0,15.4
1,1,LVI (56),23,20,1,18,6,1,20220213,14.7,5.8,0.0,25.6


## Loading Fact Table

In [34]:
set_dataframe(user_id, pwd, host_name, dst_dbname, df_superbowls, 'fact_superbowls', 'superbowl_key', 'insert')